<img src="./pictures/logo-insa.png" style="float:right; max-width: 60px; display: inline" alt="INSA" /></a>

# Sizing of a multi-rotor drone

*Written by Marc Budinger, INSA Toulouse, France*

In [15]:
((7.999e5-100)*0.225**3/6+0.5*(-203996.15+39)*0.225**2-2.81*0.225+0.106)/(2e9*3.14159/4*0.01**4)

-232.03672642674564

The objective of this notebook is to select the best compromise of components (propeller, motor, ESC, battery) of a multi-rotor drone for given specifiations.

**Scipy** and **math** packages will be used for this notebook in order to illustrate the optimization algorithms of python.

In [104]:
import scipy
import scipy.optimize
from math import pi
from math import sqrt
import math
import timeit


import numpy as np
from pyDOE import *

import pandas as pd 
from pandas import read_csv

## Sizing code

The set of equations of a sizing code can generate typical issues such : 
- Underconstrained set of equations: the lacking equations can come from additional scenarios, estimation models or additional sizing variable.   
- overconstrained equations often due to the selection of a component on multiple critera: the adding of over-sizing coefficients and constraints in the optimization problem can generally fix this issue   
- algebraic loops often due to selection criteria requiring informations generally available after the selection 

Concerning overconstraints components, we have here:
- Brushless motors with multiple torque and voltage constraints (hover and transient vertical displacement) 

Multiple algebraic loops appears in the sizing problem:
- The thrust depends of the total mass which depend of components required for generating this thrust.

The final optimization problem depends thus of these parameters:
- $\beta=pitch/diameter$ ratio to define the propeller (underconstraint)
- $k_M$ over sizing coefficient on the load mass to estimate the final total mass (algebraic loop)
- $k_{mot}$ over sizing coeffcient on the motor torque to estimate the max torque with the hover flight conditions (algebraic loop)
- $k_{speed}$ over sizing coeffcient on the motor speed to take into account voltage limits during hover or take-off flight (algebraic loop)
- $k_{ND}$ slow down propeller coef : ND = kNDmax / k_ND (underconstraint)
- $k_{frame}$ aspect ratio e/c (thickness/side) for the beam of the frame (underconstraint)
- $k_{Mb}$ over sizing coefficient on the battery load mass (underconstraint)
- $k_{vb}$ over sizing coefficient for the battery voltage (underconstraint)



More details in the setting up of sizing code can be found in the  [following paper](https://www.researchgate.net/profile/Marc_Budinger/publication/277933677_Computer-aided_definition_of_sizing_procedures_and_optimization_problems_of_mechatronic_systems/links/55969de508ae793d137c7ea5/Computer-aided-definition-of-sizing-procedures-and-optimization-problems-of-mechatronic-systems.pdf):  

> Reysset, A., Budinger, M., & Maré, J. C. (2015). Computer-aided definition of sizing procedures and optimization problems of mechatronic systems. Concurrent Engineering, 23(4), 320-332.

The sizing code is defined here in a function which can give:
- an evaluation of the objective: here the total mass or the autonomy of the battery.
- an evaluation of the constraints: must be positive to be validated.


## Objectives and specifications

Main specifications :
- a load (video, control card) of mass $M_{load}$.  
- the surface of the plate $A_{top}$ and $A_{side}$.
- an autonomy $t_{hf}$ for the hover flight.
- an acceleration to take off $a_{to}$.


In [105]:
# Specifications

# Load
M_load=4 # [kg] load mass

# Acceleration take off
a_to= 1*9.81 # [m/s²] acceleration

# Autonomy
t_h=30 # [min] time of hover fligth

# Objectif
MaxTime=True # Objective

## Architecture defintion and design assumptions

In [106]:
# Architecture of the multi-rotor drone (4,6, 8 arms, ...)
Narm=4 # [-] number of arm
Np_arm=1 # [-] number of propeller per arm (1 or 2)
Npro=Np_arm*Narm # [-] Propellers number


In [107]:
# -----------------------
# sizing code
# -----------------------
# inputs: 
# - param: optimisation variables vector (reduction ratio, oversizing coefficient)
# - arg: selection of output  
# output: 
# - objective if arg='Obj', problem characteristics if arg='Prt', constraints other else

def SizingCode(param, arg):
# Design variables
# ---
    k_M=param[0] # over sizing coefficient on the load mass 
    k_mot=param[1] # over sizing coefficient on the motor torque
    k_speed_mot=param[2] # over sizing coefficient on the motor speed
    beta=param[3] # pitch/diameter ratio of the propeller
    k_ND=param[4] # slow down propeller coef : ND = kNDmax / k_ND
    k_frame=param[5] # aspect ratio e/c (thickness/side) for the beam of the frame
    k_Mb=param[6] # over sizing coefficient on the battery load mass 
    k_vb=param[7] # over sizing coefficient for the battery voltage
    
# Hover & Take-Off thrust 
# ---
    Mtotal=k_M*M_load # [kg] Estimation of the total mass (or equivalent weight of dynamic scenario)
    
    Tpro_hover=Mtotal*(9.81)/Npro # [N] Thrust per propeller for hover
    Tpro_takeoff=Mtotal*(9.81+a_to)/Npro # [N] Thrust per propeller for take-off

# Propeller selection
# ---
    rho_air=1.18 # [kg/m^3] air density 
    
# Propeller characteristicss
# Ref : APC 

    C_t=4.27e-02 + 1.44e-01 * beta # Thrust coef with T=C_T.rho.n^2.D^4
    C_p=-1.48e-03 + 9.72e-02 * beta  # Power coef with P=C_p.rho.n^3.D^5

    NDmax=105000/60*.0254 # [Hz.m] max speed limit (N.D max)

    Dpro_ref=11*.0254 # [m] diameter
    Mpro_ref=0.53*0.0283 # [kg] mass

# Propeller selection with take-off scenario

    Dpro=(Tpro_takeoff/(C_t*rho_air*(NDmax/k_ND)**2))**0.5  # [m] Propeller diameter
    n_pro_takeoff=NDmax/k_ND/Dpro # [Hz] Propeller speed 
    #print(k_ND)
    #print(n_pro_takeoff*60, 'RPM')
    #print(n_pro_takeoff*60*Dpro/.0254, 'RPMxInch')
    Wpro_takeoff=n_pro_takeoff*2*3.14 # [rad/s] Propeller speed
    
    Mpro=Mpro_ref*(Dpro/Dpro_ref)**2 # [kg] Propeller mass
    
    Ppro_takeoff=C_p*rho_air*n_pro_takeoff**3*Dpro**5# [W] Power per propeller
    Qpro_takeoff=Ppro_takeoff/Wpro_takeoff # [N.m] Propeller torque
    #print(Qpro , 'N.m')

# Propeller torque& speed for hover

    n_pro_hover=sqrt(Tpro_hover/(C_t*rho_air*Dpro**4)) # [Hz] hover speed
    Wpro_hover=n_pro_hover*2*3.14 # [rad/s] Propeller speed    
    
    Ppro_hover=C_p*rho_air*n_pro_hover**3*Dpro**5# [W] Power per propeller
    Qpro_hover=Ppro_hover/Wpro_hover # [N.m] Propeller torque    
    
# Battery voltage estimation with propeller power
    V_bat_est=k_vb*1.84*(Ppro_takeoff)**(0.36) # [V] battery voltage estimation

# Motor selection & scaling laws
# ---
    # Selection of motor with hover scenario
    Tmot=k_mot*Qpro_hover   # [N.m] Motor nominal torque per propeller
        
    # Motor reference
    # Ref : AXI 5325/16 GOLD LINE
    Tmot_ref=2.32  # [N.m] rated torque
    Tmot_max_ref=85/70*Tmot_ref # [N.m] max torque
    Rmot_ref=0.03  # [Ohm] resistance
    Mmot_ref=0.575 # [kg] mass
    Ktmot_ref=0.03 # [N.m/A] torque coefficient
    Tfmot_ref=0.03 # [N.m] friction torque (zero load, nominal speed) 
    
    Mmot=Mmot_ref*(Tmot/Tmot_ref)**(3/3.5) # [kg] Motor mass
    Tmot_max=Tmot_max_ref*(Tmot/Tmot_ref)**(1) # [N.m] max torque
    
    # Selection with take-off speed
    Ktmot=V_bat_est/(k_speed_mot*Wpro_takeoff) # [N.m/A] or [V/(rad/s)] Kt motor 
    Rmot=Rmot_ref*(Tmot/Tmot_ref)**(-5/3.5)*(Ktmot/Ktmot_ref)**(2)  # [Ohm] motor resistance
    Tfmot=Tfmot_ref*(Tmot/Tmot_ref)**(3/3.5) # [N.m] Friction torque

    # Hover current and voltage
    Imot_hover = (Qpro_hover+Tfmot)/Ktmot # [I] Current of the motor per propeller
    Umot_hover = Rmot*Imot_hover + Wpro_hover*Ktmot # [V] Voltage of the motor per propeller
    P_el_hover=Umot_hover*Imot_hover # [W] Hover : electrical power
    
    # Take-Off current and voltage
    Imot_takeoff = (Qpro_takeoff+Tfmot)/Ktmot # [I] Current of the motor per propeller
    Umot_takeoff = Rmot*Imot_takeoff + Wpro_takeoff*Ktmot # [V] Voltage of the motor per propeller
    P_el_takeoff=Umot_takeoff*Imot_takeoff # [W] Takeoff : electrical power
    
   
# Battery selection & scaling laws  
# --- 
    # Battery
    # Ref : MK-quadro
    Mbat_ref=.329 # [kg] mass
    #Ebat_ref=4*3.7*3.3*3600 # [J] energy
    Ebat_ref=220*3600*.329 # [J]
#    print(V_bat_est)
    Ncel=V_bat_est/3.7# [-] Cell number, round (up value)
    V_bat=3.7*Ncel # [V] Battery voltage
    
    Mbat=k_Mb*M_load # Battery mass
    
    # Hover --> autonomy
    Ebat = Ebat_ref*Mbat/Mbat_ref*.8 # [J] Energy  of the battery (.8 coefficient because 80% use only of the total capacity)
    C_bat = Ebat/V_bat # [A.s] Capacity  of the battery 
    I_bat = (P_el_hover*Npro)/.95/V_bat # [I] Current of the battery
    t_hf = C_bat/I_bat/60 # [min] Hover time 
    
   
# ESC selection
# ---

# ESC
# Ref : Turnigy K_Force 70HV 
    Pesc_ref=3108 # [W] Power
    Mesc_ref=.115 # [kg] Mass
    
    P_esc=P_el_takeoff*V_bat_est/Umot_takeoff # [W] power electronic power (corner power or apparent power)
    
    Mesc = Mesc_ref*(P_esc/Pesc_ref) # [kg] Mass ESC
    Vesc=1.84*P_esc**(0.36)# [V] ESC voltage
         
# Frame
# ---
    # Length calculation   
    sep= 2*pi/Narm #[rad] interior angle separation between propellers
    Lb = Dpro/(2*scipy.sin(sep/2)) #[m] length of the arm

    # Static stress
    # Sigma_max=200e6/4 # [Pa] Alu max stress (2 reduction for dynamic, 2 reduction for stress concentration)
    Sigma_max=280e6/4 # [Pa] Composite max stress (2 reduction for dynamic, 2 reduction for stress concentration)
    
    # Tube diameter & thickness
    Dfra = (Tpro_takeoff*Np_arm/Sigma_max*Lb*32/(pi*(1-(1-2*k_frame)**4)))**(1/3)  # [m] side of the section of the beam 
    Efra= k_frame*Dfra # [m] thickness of side 
          
    # Mass
    # Mfra=pi/4*(Dfra**2-(Dfra-2*Efra)**2)*Lb*2700 # [kg] mass of the frame (beams only) 
    Mfra=pi/4*(Dfra**2-(Dfra-2*Efra)**2)*Lb*1700 # [kg] mass of the frame (beams only)  composite
        
      
# Objective and Constraints sum up
# ---
    Mtotal_final = (Mesc+Mpro+Mmot)*Npro+M_load+Mbat+Mfra*Narm
    #Tmot_hover=Tfmot+Qpro 
    #k_surf_real=(pi*Dpro**2/4-Afra)/(pi*Dpro**2/4)

    if MaxTime==True:
        constraints = [(Mtotal-Mtotal_final)/Mtotal_final,(V_bat-Umot_takeoff)/V_bat, (Tmot_max-Qpro_takeoff)/Qpro_takeoff, (V_bat-Vesc)/V_bat]
    else:
        constraints = [(Mtotal-Mtotal_final)/Mtotal_final,(V_bat-Umot_takeoff)/V_bat, (Tmot_max-Qpro_takeoff)/Qpro_takeoff, (V_bat-Vesc)/V_bat, (t_hf-t_h)/t_h]
    
# Objective and contraints
    if arg=='Obj':
        if MaxTime==True:
            return 1/t_hf # for time maximisation
        else:
            return (Mtotal+Mtotal_final)/2 # for mass optimisation 
            
        
    elif arg=='Prt':
        print("* Specifications:")
        print("           Mass of load : %.2f kg"% M_load)
        print("           Take off acceleration : %.2f g"%(a_to/9.81))
        print("* Optimisation objective:")
        print("           Max Autonomy : %.1f min"%t_hf)
        print("* Optimisation variables:")
        print("           beta angle consisting of pitch /diameter = %.2f"% beta)
        print("           oversizing coefficient on the load mass k_M = %.2f"% k_M)
        print("           Ratio for battery mass = %.2f"%k_Mb)
        print("           oversizing coefficient on the motor torque k_mot = %.2f"%k_mot)
        print("           oversizing coefficient on the motor speed k_speed_mot = %.2f"%k_speed_mot)
        print("           undersizing coefficient on the propeller speed k_ND = %.2f"%(k_ND))
        print("           aspect ratio thickness/side k_frame = %.2f"%k_frame)
        print("           over sizing coefficient on the battery load mass = %.2f"%k_Mb)
        print("           over sizing coefficient for the battery voltage = %.2f"%k_vb) 
        print("* Architecture description:")
        print("           Numbers of arms = ",Narm)        
        print("           Numbers of propellers = ",Npro)        
        print("")
        print("* Mass values:")
        print("           Total mass: %.3f kg"%(Mtotal_final))
        print("           Propeller mass (1x): %.3f kg"%(Mpro))
        print("           Motor mass (1x): %.3f kg"%(Mmot))
        print("           Battery mass: %.3f kg"%(Mbat))
        print("           ESC mass per propeller : %.3f kg"%(Mesc))
        print("           Arm mass (1x) : %.3f kg"%(Mfra))
        print("")
        print("* Frame size:")
        print("           Beam diameter = %.2f mm" %(Dfra*1000))
        print("           Thickness = %.2f mm" % (Efra*1000))
        print("           Length of the arm = %.2f mm" % (Lb*1000))
        print("           Interior angle / separation between propellers = %.2f °" % (sep*180/pi))        
        print("")
        print("* Propellers:")
        print("           NxD takeoff = %.0f RPMxInch"%(n_pro_takeoff*60*Dpro/.0254))
        print("           Diameter Dpro = %.2g m"%Dpro)
        print("           Pitch  = %.2g m"%(beta*Dpro))
#        print("           Ratio of useful area = %.2f %%"%(k_surf_real*100))
        print("           Power coefficient C_p: - for statics: %.4f "%(C_p))
        print("")
        print("           Thrust coefficient C_t: - for statics: %.4f "%(C_t))
        print("")
        print("           Rotational speed:      - for hover: %.0f RPM "%(Wpro_hover*60/2/pi))
        print("                                  - for vertical acceleration: %.0f RPM "%(Wpro_takeoff*60/2/pi))
        print("           Interior angle / separation between propellers: %.0f ° "%(sep*180/pi))
        print("")
        print("* Thrust force per propeller:")
        print("           Hover: %.2f N"%(Tpro_hover))
        print("           Vertical + Acceleration (2g): %.2f  N"%(Tpro_takeoff))
        print("")
        print("* Power per propeller:")
        print("           Hover power: %.2f W"%(Ppro_hover))
        print("           Vertical + Acceleration (2g): %.2f  W"%(Ppro_takeoff))
        print("")
        print("* Torque per propeller:")
        print("           Hover : %.3f N.m"%(Tpro_hover))
        print("           Vertical + Acceleration (2g): %.3f  N.m"%(Tpro_takeoff))
        print("")
        print("* Voltage:")
        print("           Motor voltage Umot: %.2f V"%(Umot_hover))
        print("           Transient voltage in vertical + Acceleration (2g): U_ver: %.2f V"%(Umot_takeoff))
        print("")
        print("* Current:")
        print("           Motor current Hover: %.2f A"%(Imot_hover))
        print("           Transient current in vertical + Acceleration (2g): I_ver: %.2f A"%(Imot_takeoff))
        print("")
        print("* Battery:")
        print("           Battery capacity = %.2f A.h" %(C_bat/3600))
        print("           Battery voltage = %.2f V" %(V_bat))
        print("           Battery voltage estimated= %.2f V" %(V_bat_est))
        print("           ESC voltage = %.2f V" %(Vesc))
        print("")
        print("* Normalized constraints (should be >0):")
        print("           Estimated mass - Total final mass / Total final mass =%.2f  " %constraints[0])
        print("           (V_bat-Umot_takeoff) / V_bat = %.3f"%constraints[1])   
        print("           (Tmot_max-Qpro_takeoff)/Qpro_takeoff = %.3f "%constraints[2])
        print("           (V_bat-Vesc)/V_bat = %.3f "%constraints[3])
        if MaxTime==False:
            print("           (T_hf-T_h)/T_h = %.3f m"%constraints[4])

    else:
        return constraints


## Optimization problem


We will now use the [opmization algorithms](https://docs.scipy.org/doc/scipy/reference/optimize.html) of the Scipy package to solve and optimize the configuration. We use here the SLQP algorithm without explicit expression of the gradient (Jacobian). A course on Multidisplinary Gradient optimization algorithms and gradient optimization algorithm is given [here](http://mdolab.engin.umich.edu/sites/default/files/Martins-MDO-course-notes.pdf):
> Joaquim R. R. A. Martins (2012). A Short Course on Multidisciplinary Design Optimization. Univeristy of Michigan


The first step is to deduce the initial values of optimisation variables, which minimize the contraints. For that purpose, we will perform a ranomized design based on a latin-hypercube, which will give us initial values. For these values, the contraints are calculated and only the constraint for a minimum objective is saved.

In [108]:
def MultiStart_SLSQP(objectif, xlimits, contraints,iteration,accuracy,samples):
   
#1: DoE for the initial conditions
#------------    
    minimum=xlimits[:,0] #lower limit contraints
    maximum=xlimits[:,1] #upper limit contraints
    diff=maximum-minimum 

    design=lhs(len(minimum), samples) # doe for X-samples distributed along 0 and 1.

    doe = [vec *diff+minimum for vec in design] # doe transformed to our array of initial parameters 

    
    
# 2 : slsqp optimizing loop: find the minimum objective and its parameter vector
#------------    
    
    obj=[] # vector to save objectives which constraints are positive


    for vector in doe: #slsqp algorithm for the random parameters
                
        result = scipy.optimize.fmin_slsqp(func=objectif, x0=vector,bounds=xlimits,
                                           f_ieqcons=contraints, iter=iteration,
                                           acc=accuracy, iprint =-1)

        if all (value>0 for value in contraints(result)): #only consider positive contraints!
            
            obj.append(objectif(result)) #saving objective with positive contraints

            if min(obj)==objectif(result): #objectif is examined if it is the minimum mass from the vector obj
                min_vector=result #choose minimum objective
    return min_vector

In [109]:
smpl=100 #samples
lims = np.array([
[1, 20],# k_M, over sizing coefficient on the load mass 
[1, 10],# k_mot, over sizing coefficient on the motor torque
[1, 10],# k_speed_mot, adaption winding coef on the motor speed
[0.3, 0.6], # beta, pitch/diameter ratio of the propeller
[1, 10], # k_ND, reduction of product speed.diameter on the propeller
[0.05,.5], # k_frame, aspect ratio e/c (thickness/side) for the beam of the frame
[.2, 15], # k_Mb, ratio battery/load mass
[1, 5],# k_vb, oversizing coefficient for voltage evaluation
]) # ranges of contraints

iter=1500
acc=1e-8

# optimization with SLSQP algorithm
contrainte=lambda x: SizingCode(x, 'Const') # calling functions from the SizingCode
objectif=lambda x: SizingCode(x, 'Obj') # calling functions from the SizingCode

min_vec=MultiStart_SLSQP(objectif,lims,contrainte,iter,acc, smpl)

Accuracy and the number of iteration are two relevant factors that give precision to the final result. The smaller the accuracy rate is and the more iterations are completed, the less deviation will be in the objective. Constraints tend also to converge to 0 if the adequate relationship is expressed. A compromise must be seeked between high accuracy and time resolution .


In [110]:

print("-----------------------------------------------")
print("Final characteristics after optimization :")

SizingCode(min_vec, 'Prt')

print("-----------------------------------------------")


-----------------------------------------------
Final characteristics after optimization :
* Specifications:
           Mass of load : 14.00 kg
           Take off acceleration : 1.00 g
* Optimisation objective:
           Max Autonomy : 50.9 min
* Optimisation variables:
           beta angle consisting of pitch /diameter = 0.30
           oversizing coefficient on the load mass k_M = 9.36
           Ratio for battery mass = 4.93
           oversizing coefficient on the motor torque k_mot = 1.65
           oversizing coefficient on the motor speed k_speed_mot = 1.22
           undersizing coefficient on the propeller speed k_ND = 1.12
           aspect ratio thickness/side k_frame = 0.05
           over sizing coefficient on the battery load mass = 4.93
           over sizing coefficient for the battery voltage = 2.70
* Architecture description:
           Numbers of arms =  8
           Numbers of propellers =  8

* Mass values:
           Total mass: 130.993 kg
           Propeller 